# Imports

In [49]:
import pandas as pd
import numpy as np
import requests
from PyPDF2 import PdfFileReader, PdfFileMerger, PdfFileWriter
import PyPDF2
from bs4 import BeautifulSoup
import os
import shutil
from urllib.parse import urljoin
from urllib.request import Request, urlopen
from io import StringIO, BytesIO
import cloudinary.uploader
import json
from datetime import date
import sys

# Testing own api

In [32]:
url = 'http://bank-price-api.herokuapp.com/'
post = {"1":{"url":"https://www.abanca.pt", 
                    "name":"ABANCA Corporación Bancaria, S.A.", 
                    "num_pdfs":"3", 
                    "last_updated":"210401", 
                    "sum_sizes":"40", 
                    "bp_bank_id":"0170"}
             ,
              "2":{"url":"https://www.bankinter.pt", 
                    "name":"Bankinter, S.A.", 
                    "num_pdfs":"3", 
                    "last_updated":"210401", 
                    "sum_sizes":"40", 
                    "bp_bank_id":"0269"}
              ,
              "3":{"url":"https://www.bancobic.ao", 
                    "name":"Banco BIC, SA", 
                    "num_pdfs":"3", 
                    "last_updated":"210401", 
                    "sum_sizes":"40", 
                    "bp_bank_id":"0079"}
              }


In [30]:
post

{'1': {'url': 'https://www.abanca.pt',
  'name': 'ABANCA Corporación Bancaria, S.A.',
  'num_pdfs': '3',
  'last_updated': '210401',
  'sum_sizes': '40',
  'bp_bank_id': '0170'},
 '2': {'url': 'https://www.bankinter.pt',
  'name': 'Bankinter, S.A.',
  'num_pdfs': '3',
  'last_updated': '210401',
  'sum_sizes': '40',
  'bp_bank_id': '0269'}}

In [31]:
r = requests.post(url, data=post)
r

<Response [405]>

In [63]:
## expected result
result = {
    "1": {
        "bp_bank_id": "0170",
        "bp_pdf_url": "https://clientebancario.bportugal.pt/sites/default/files/precario/0170_/0170_PRE.pdf",
        "last_updated": "210401",
        "list_pdfs": [
            "https://www.abanca.pt/files/documents/precario-folheto-comissoes-823a7663.pdf",
            "https://www.abanca.pt/files/documents/folheto-taxa-juro-precario-5890906f.pdf",
            "https://www.abanca.pt/files/documents/precariodevaloresmobiliarios-23c714a2.pdf",
            "https://www.abanca.pt/files/documents/glossario-6479e89f.pdf",
            "https://www.abanca.pt/files/docs/leaflet-your-rights-payments-eu.pdf"
        ],
        "name": "ABANCA Corporación Bancaria, S.A.",
        "num_pdfs": 5,
        "price_page": "https://www.abanca.pt/pt/precario/",
        "sum_sizes": "40",
        "url": "https://www.abanca.pt"
    },
    "2": {
        "bp_bank_id": "0269",
        "bp_pdf_url": "https://clientebancario.bportugal.pt/sites/default/files/precario/0269_/0269_PRE.pdf",
        "last_updated": "210401",
        "list_pdfs": [
            "https://www.bankinter.pt/documents/20182/115991/Pre%C3%A7%C3%A1rio+Completo+TFSI++20210401.pdf/c489f0cd-3831-4441-812e-f351741d2b22",
            "https://banco.bankinter.pt/particulares/pdfs/precario/folha_rosto.pdf",
            "https://banco.bankinter.pt/particulares/pdfs/precario/informacao_geral.pdf",
            "https://banco.bankinter.pt/particulares/pdfs/precario/informacao_complementar.pdf",
            "https://banco.bankinter.pt/particulares/pdfs/precario/p_fcd_contas_deposito.pdf",
            "https://banco.bankinter.pt/particulares/pdfs/precario/p_fcd_operacoes_credito.pdf",
            "https://banco.bankinter.pt/particulares/pdfs/precario/p_fcd_cartoes_credito_debito.pdf",
            "https://banco.bankinter.pt/particulares/pdfs/precario/p_fcd_cheques.pdf",
            "https://banco.bankinter.pt/particulares/pdfs/precario/p_fcd_transferencias.pdf",
            "https://banco.bankinter.pt/particulares/pdfs/precario/p_fcd_cobrancas.pdf",
            "https://banco.bankinter.pt/particulares/pdfs/precario/p_fcd_prestacao_servicos.pdf",
            "https://banco.bankinter.pt/particulares/pdfs/precario/o_fcd_contas_deposito.pdf",
            "https://banco.bankinter.pt/particulares/pdfs/precario/o_fcd_operacoes_credito.pdf",
            "https://banco.bankinter.pt/particulares/pdfs/precario/o_fcd_cartoes_credito_debito.pdf",
            "https://banco.bankinter.pt/particulares/pdfs/precario/o_fcd_cheques.pdf",
            "https://banco.bankinter.pt/particulares/pdfs/precario/o_fcd_transferencias.pdf",
            "https://banco.bankinter.pt/particulares/pdfs/precario/o_fcd_cobrancas.pdf",
            "https://banco.bankinter.pt/particulares/pdfs/precario/o_fcd_prestacao_servicos.pdf",
            "https://banco.bankinter.pt/particulares/pdfs/precario/o_fcd_operacoes_estrangeiro.pdf",
            "https://banco.bankinter.pt/particulares/pdfs/precario/p_ftj_contas_deposito.pdf",
            "https://banco.bankinter.pt/particulares/pdfs/precario/p_ftj_operacoes_credito.pdf",
            "https://banco.bankinter.pt/particulares/pdfs/precario/o_ftj_contas_deposito.pdf",
            "https://banco.bankinter.pt/particulares/pdfs/precario/o_ftj_operacoes_credito.pdf",
            "https://banco.bankinter.pt/particulares/pdfs/precario/ptfs_a.pdf",
            "https://banco.bankinter.pt/particulares/pdfs/precario/ptfs_b.pdf",
            "https://banco.bankinter.pt/particulares/pdfs/precario/ptfs_c.pdf",
            "https://banco.bankinter.pt/particulares/pdfs/precario/ptfs_d.pdf"
        ],
        "name": "Bankinter, S.A.",
        "num_pdfs": 27,
        "price_page": "https://www.bankinter.pt/precario1",
        "sum_sizes": "40",
        "url": "https://www.bankinter.pt"
    },
    "3": {
        "bp_bank_id": "0079",
        "bp_pdf_url": "https://clientebancario.bportugal.pt/sites/default/files/precario/0079_/0079_PRE.pdf",
        "last_updated": "210401",
        "list_pdfs": [],
        "name": "Banco BIC, SA",
        "num_pdfs": 0,
        "price_page": {
            "error": "page not reachable"
        },
        "sum_sizes": "40",
        "url": "https://www.bancobic.ao"
    }
}

# last_updated return

In [34]:
pdf = "https://banco.bankinter.pt/particulares/pdfs/precario/p_fcd_contas_deposito.pdf"
remoteFile = urlopen(Request(pdf)).read()

In [35]:
memoryFile = BytesIO(remoteFile)
pdfFile = PdfFileReader(memoryFile)

In [39]:
pdfFile.getDocumentInfo()

{'/Author': 'BK10813',
 '/CreationDate': "D:20210331170537+01'00'",
 '/ModDate': "D:20210331170537+01'00'",
 '/Producer': 'Microsoft: Print To PDF',
 '/Title': '01. Contas de DepÃ³sito - FCD - Particulares - 20210401.pdf'}

In [61]:
sys.getsizeof(remoteFile)

721022

In [67]:
for bank_id, vals in result.items():
    pdfs = vals.get('list_pdfs')
    if pdfs == []:
        break
    else:
        filesizes = []
        for pdf in pdfs:
            try:
                remote = urlopen(Request(pdf)).read()
                memory = BytesIO(remote)
                file = PdfFileReader(memory)
                filesizes.append(sys.getsizeof(memory))
            except:
                print(f'cannot reach: {pdf}')

        vals['sum_sizes'] = sum(filesizes)


cannot reach: https://www.abanca.pt/files/documents/glossario-6479e89f.pdf


In [68]:
result

{'1': {'bp_bank_id': '0170',
  'bp_pdf_url': 'https://clientebancario.bportugal.pt/sites/default/files/precario/0170_/0170_PRE.pdf',
  'last_updated': '210401',
  'list_pdfs': ['https://www.abanca.pt/files/documents/precario-folheto-comissoes-823a7663.pdf',
   'https://www.abanca.pt/files/documents/folheto-taxa-juro-precario-5890906f.pdf',
   'https://www.abanca.pt/files/documents/precariodevaloresmobiliarios-23c714a2.pdf',
   'https://www.abanca.pt/files/documents/glossario-6479e89f.pdf',
   'https://www.abanca.pt/files/docs/leaflet-your-rights-payments-eu.pdf'],
  'name': 'ABANCA Corporación Bancaria, S.A.',
  'num_pdfs': 5,
  'price_page': 'https://www.abanca.pt/pt/precario/',
  'sum_sizes': 320,
  'url': 'https://www.abanca.pt'},
 '2': {'bp_bank_id': '0269',
  'bp_pdf_url': 'https://clientebancario.bportugal.pt/sites/default/files/precario/0269_/0269_PRE.pdf',
  'last_updated': '210401',
  'list_pdfs': ['https://www.bankinter.pt/documents/20182/115991/Pre%C3%A7%C3%A1rio+Completo+TF